In [20]:
import pandas as pd

iterations=[1, 2, 3]
delays=[0.016, 0.025, 0.05, 0.1, 0.2]


# Get the start and end files and create another file with the latency in milliseconds

for delay in delays:

    for i in iterations:

        df_start = None
        df_end = None
        df = None

        df_start = pd.read_csv("../DATA/results_latency_nodriver/batch"+str(delay)+"_"+str(i)+"-start.txt", header=None, names=["hash", "nano-start"])

        df_end = pd.read_csv("../DATA/results_latency_nodriver/batch"+str(delay)+"_"+str(i)+"-end.txt", header=None, names=["hash", "nano-end"])


        ### MERGE DATAFRAMES id_start and id_end
        df = df_start.merge(df_end, left_on=['hash'], right_on=['hash'], how='outer')
        if not df.isnull().sum().sum() == 0:
            print("some hash do not coincide")
        df.dropna(inplace=True)

        df["processing_time"] = df["nano-end"].astype(float) - df["nano-start"].astype(float)
        df["processing_time_ms"] = df["processing_time"]/1000000

        df.to_csv("../DATA/results_latency_nodriver/processed/latency"+str(delay)+"_"+str(i)+".csv", index=False)

df.head()

0.016 1


,hash,nano-start,nano-end,processing_time,processing_time_ms
0,-1903027235,38803975920848939,38803976211926280,291077344.0,291.077344
1,222079332,38803976009917730,38803976303460606,293542880.0,293.542880
2,-1098426718,38803976010560312,38803976303826516,293266200.0,293.266200
3,2032994932,38803976015178314,38803976303997127,288818816.0,288.818816
4,-1512773053,38803976015902476,38803976304162143,288259664.0,288.259664


In [61]:
delays=[0.016, 0.025, 0.05, 0.1, 0.2]


for delay in delays:

    df1 = pd.read_csv("../DATA/results_latency_nodriver/processed/latency"+str(delay)+"_"+str(1)+".csv")
    df2 = pd.read_csv("../DATA/results_latency_nodriver/processed/latency"+str(delay)+"_"+str(2)+".csv")
    df3 = pd.read_csv("../DATA/results_latency_nodriver/processed/latency"+str(delay)+"_"+str(3)+".csv")

    df = df1.merge(df2[["hash", "processing_time_ms"]], left_on=['hash'], right_on=['hash'], how='outer')
    df = df.merge(df3[["hash", "processing_time_ms"]], left_on=['hash'], right_on=['hash'], how='outer')
    if not df.isnull().sum().sum() == 0:
        print("some hash do not coincide")
    df.dropna(inplace=True)

    df["processing_time_ms"] = (df["processing_time_ms"]+df["processing_time_ms_x"]+df["processing_time_ms_y"])/3
    df.drop(['nano-start', 'nano-end', 'processing_time', 'processing_time_ms_x', 'processing_time_ms_y'], axis='columns', inplace=True)

    if len(df1) != 1250 or len(df2) != 1250 or len(df3) != 1250:
        print("Something may be wrong with this batches")

    df.to_csv("../DATA/results_latency_nodriver/processed/processed_averaged/latency"+str(delay)+".csv", index=False)